In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd /content/drive/MyDrive/690F

/content/drive/MyDrive/690F


In [1]:
import string
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import pickle
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv("final_processed_train_data.csv")

In [3]:
train_df.head()

,Unnamed: 0,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,...,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,comment_text_tokenized,prot_word_count,edited_comment_text
0,0,59848,0.000000,This cool Its like want mother read Really gre...,0.000000,0.0,0.000000,0.00000,0.0,NaN,...,0,0,0,0,0.0,0,4,"['this', 'cool', 'its', 'like', 'want', 'mothe...",0,this cool its like mother read really great idea
1,1,59849,0.000000,Thank This life lot anxietyinducing Keep let way,0.000000,0.0,0.000000,0.00000,0.0,NaN,...,0,0,0,0,0.0,0,4,"['thank', 'this', 'life', 'lot', 'anxietyinduc...",0,thank this life lot anxietyindcing keep let way
2,2,59852,0.000000,This urgent design problem kudos taking Very i...,0.000000,0.0,0.000000,0.00000,0.0,NaN,...,0,0,0,0,0.0,0,4,"['this', 'urgent', 'design', 'problem', 'kudos...",0,this rgent design problem kdos taking very imp...
3,3,59855,0.000000,Is I able install site When releasing,0.000000,0.0,0.000000,0.00000,0.0,NaN,...,0,0,0,0,0.0,0,4,"['is', 'i', 'able', 'install', 'site', 'when',...",0,is i able install site when releasing
4,4,59856,0.893617,haha guys bunch losers,0.021277,0.0,0.021277,0.87234,0.0,0.0,...,0,0,1,0,0.0,4,47,"['haha', 'guys', 'bunch', 'losers']",0,haha gys bnch losers


In [4]:
train_df.columns

Index(['Unnamed: 0', 'id', 'target', 'comment_text', 'severe_toxicity',
       'obscene', 'identity_attack', 'insult', 'threat', 'asian', 'atheist',
       'bisexual', 'black', 'buddhist', 'christian', 'female', 'heterosexual',
       'hindu', 'homosexual_gay_or_lesbian',
       'intellectual_or_learning_disability', 'jewish', 'latino', 'male',
       'muslim', 'other_disability', 'other_gender', 'other_race_or_ethnicity',
       'other_religion', 'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count',
       'comment_text_tokenized', 'prot_word_count', 'edited_comment_text'],
      dtype='object')

In [5]:

train_df.dropna(inplace = True)
to_show = ['id', 'target',
       'comment_text', 'asian', 'atheist', 'bisexual', 'black', 'buddhist',
       'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white']
train_df.head().loc[:, to_show]

,id,target,comment_text,asian,atheist,bisexual,black,buddhist,christian,female,...,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white
11,239579,0.4400,This great story Man I wonder person yelled sh...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,239607,0.9125,Yet Muslims acts pilloried So okay smear entir...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,239644,0.0000,Because people drive cars ones cause wear tear...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58,239653,0.3000,Mormons complicated relationship federal law,0.0,0.0,0.0,0.0,0.0,0.2,0.0,...,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0
111,239744,0.0000,I thing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
train_df = train_df[train_df['target'] >= 0.20]

In [7]:
# the number of non toxic comments was more than 10 times the number of toxic comments, we have subsampled it to the following distribution
print("Number of non toxic comments: "+str(len(train_df[train_df["target"] < 0.5])))
print("Number of toxic comments: "+str(len(train_df[train_df["target"] >= 0.5])))

Number of non toxic comments: 41566
Number of toxic comments: 25142


In [8]:
train_df["prot_word_count"][train_df["prot_word_count"] > 0] = 1

In [9]:
print("Number of comments with bias inducing words not present: " + str(len(train_df[train_df["prot_word_count"] == 0])))
print("Number of comments with bias inducing words present: " + str(len(train_df[train_df["prot_word_count"] == 1])))

Number of comments with bias inducing words not present: 28700
Number of comments with bias inducing words present: 38008


In [10]:
X = train_df[["comment_text", "prot_word_count", "edited_comment_text"]]
y = np.array(train_df['target'].apply(lambda x: x > .5).astype('int'))



In [11]:
prot_attr = ['transgendered', 'sex', 'bigotry', 'racist', 'muslim', 'racism', 'priest', 'black', 'sexual', 'latino', 'man', 'islam', 'gay', 'men', 'straight', 'transgender', 'homosexual', 'jewish', 'catholic', 'heterosexual', 'women', 'church', 'religious', 'military', 'asian', 'hindu', 'buddhist', 'religion', 'atheist', 'white', 'chinese', 'male', 'christian', 'race', 'bisexual', 'jew', 'woman']

In [12]:
# Standard Learning without Z

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
Xvec = vectorizer.fit_transform(X['edited_comment_text'])
X_vect = pd.DataFrame(Xvec.todense(), columns=vectorizer.get_feature_names_out())

In [15]:
word_counts = {}
cols_to_drop = []
for i in list(X_vect.columns):
    X_vect[i][X_vect[i] > 0] = 1
    col_total = X_vect[i].sum()
    word_counts[i] = col_total
    if col_total < 11:
        cols_to_drop.append(i)

print("colstodrop",len(cols_to_drop))
print("orig_cols",len(X_vect.columns))
X_vect.drop(cols_to_drop, axis=1, inplace = True)
print("new_cols",len(X_vect.columns))

colstodrop 63444
orig_cols 76943
new_cols 13499


In [16]:
X_vect.to_csv("edited_comment_X_vect.csv")

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size=.4, stratify = y)

In [18]:
from sklearn.linear_model import LogisticRegression
# Fitting a simple Logistic Regression on tf-idf
lr = LogisticRegression()
lr.fit(X_train, y_train)

/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [19]:
print("Accuracy: "+str(lr.score(X_test, y_test)))
y_preds = lr.predict(X_test)



Accuracy: 0.7793434267725978


In [22]:
coeffs_ml_without_z = pd.DataFrame({"Feature" : X_vect.columns.tolist(),"Coefficients" : lr.coef_[0]})

In [26]:
coeffs_ml_without_z[coeffs_ml_without_z['Feature'].isin(prot_attr)]

,Feature,Coefficients
744,asian,-0.315275
827,atheist,0.172630
1788,catholic,-0.491175
1983,chinese,0.240093
2011,christian,-0.057505
6493,islam,0.273566
6605,jew,0.486719
6608,jewish,-0.133100
6892,latino,0.410707
7520,men,0.204320


In [27]:
res = coeffs_ml_without_z[coeffs_ml_without_z['Feature'].isin(prot_attr)]
res.to_csv("coeffs_ml_without_z.csv")

In [28]:
groups = {
'religion' : ['religion', 'religious', 'atheist', 'buddhist', 'christian', 'hindu', 'jewish', 'muslim', 'catholic', 'church', 'islam', 'jew', 'priest'],
'race' : ['asian', 'black', 'latino', 'white', 'chinese', 'race', 'racism'],
'gender' : ['female', 'male', 'transgender', 'man', 'men', 'trangendered', 'woman', 'women'],
'sexual_identity' : ['bisexual', 'heterosexual', 'gay', 'homosexual', 'sex', 'sexual', 'straight' ],
'misc' : ['bigotry', 'military']
}

avg_coeffs = {}

for group in groups:
    avg_coeff = np.mean(np.array(coeffs_ml_without_z[coeffs_ml_without_z['Feature'].isin(groups[group])]['Coefficients']))
    avg_coeffs[group] = avg_coeff

avg_coeffs
    

{'religion': 0.004348001324842509,
 'race': 0.11184151860766274,
 'gender': 0.20431975186541906,
 'sexual_identity': -0.06593569425689184,
 'misc': -0.16728225904961355}